In [55]:
#using randomizedsearchCV takes much less time
#try using MOE? how?

#begin developing structure of the presentation and beef up on what the hell XGBoost, QWK, fmin_powell, and the rest are doing


from sklearn.grid_search import GridSearchCV, RandomizedSearchCV
import numpy as np
import pandas as pd
from scipy.stats import randint
from xgboost import XGBClassifier, XGBRegressor
from time import time
from operator import itemgetter



DATA_DIR = '/Users/patrickkennedy/Desktop'
train = pd.read_csv(DATA_DIR + '/Project DATA/train.csv')
test = pd.read_csv(DATA_DIR + '/Project DATA/test.csv')

train['Product_Info_2'] = pd.factorize(train['Product_Info_2'])[0]
test['Product_Info_2'] = pd.factorize(test['Product_Info_2'])[0]

train.fillna(-1, inplace=True)
test.fillna(-1, inplace=True)


features = train.columns
features = features.drop("Id")
features = features.drop("Response")
X = train[features]
y = train["Response"]

print X.shape, y.shape


X_test = test[features]

dev_cutoff = len(Y) * 4/5
X_dev = X[:dev_cutoff]
y_dev = y[:dev_cutoff]
X_test = X[dev_cutoff:]
y_test = y[dev_cutoff:]






#using something other than randint... we don't want to round numbers here (for some cases)

param_dist = {"objective": ["reg:linear", "count:poisson", "multi:softmax"],
                      "learning_rate": [0.001, 0.01, 0.03],
                      "gamma": randint(0,100),
                      "max_delta_step": randint(0,10),
                      "min_child_weight": randint(1,100),
                      "subsample": randint(0.25, 0.99),
                      "colsample_bytree": randint(.5, .99),
                      #"silent":1,
                      #"num_classes":8,
                      "max_depth": randint(3,12),
                      "n_estimators": randint(5,1000)
                      }
    
    
clf = XGBRegressor(silent=True)





# Utility function to report best scores
def report(grid_scores, n_top=3):
    top_scores = sorted(grid_scores, key=itemgetter(1), reverse=True)[:n_top]
    for i, score in enumerate(top_scores):
        print("Model with rank: {0}".format(i + 1))
        print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
              score.mean_validation_score,
              np.std(score.cv_validation_scores)))
        print("Parameters: {0}".format(score.parameters))
        print("")
        
        
        

# run randomized search
n_iter_search = 50
random_search = RandomizedSearchCV(clf, param_distributions=param_dist,
                                   n_iter=n_iter_search, cv=3)

start = time()


random_search.fit(X_dev, y_dev)
print("RandomizedSearchCV took %.2f seconds for %d candidates"
      " parameter settings." % ((time() - start), n_iter_search))
report(random_search.grid_scores_)

#time 512 seconds, 2 iterations, two loss measures, two learning rates
#now trying, 50 iterations, 2x3x100x10x100x3x4x17x995, cv=3 ... low side: 12800~3hrs, high side: 9+hrs

#also think about doing param tuning for PCA... i read this somewhere today, i just forgot where


#with clf = XGBClassifier
#RandomizedSearchCV took 87973 seconds (1466 minutes / 24 hours) for a mean validation score of 0.580 (std: 0.002)
#params{'colsample_bytree':0.9, 'learning_rate': 0.03, 'max_delta_step': 8, 'min_child_weight':44, 'n_estimators':224,
        #'subsample':0.75, 'objective':'count:poisson', 'max_depth':6, 'gamma':6}
    
#Model with rank: 2
#Mean validation score: 0.569 (std: 0.002)
#Parameters: {'colsample_bytree': 0.9, 'learning_rate': 0.03, 'max_delta_step': 7, 'min_child_weight': 61, 'n_estimators': 846, 'subsample': 0.75, 'objective': 'count:poisson', 'max_depth': 4, 'gamma': 15}

#Model with rank: 3
#Mean validation score: 0.566 (std: 0.001)
#Parameters: {'colsample_bytree': 0.9, 'learning_rate': 0.01, 'max_delta_step': 4, 'min_child_weight': 77, 'n_estimators': 523, 'subsample': 0.75, 'objective': 'reg:linear', 'max_depth': 15, 'gamma': 15}



#now let's try with multi-softmax
#RandomizedSearchCV took 112468.81 seconds for 50 candidates parameter settings.
#Model with rank: 1
#Mean validation score: 0.579 (std: 0.002)
#Parameters: {'colsample_bytree': 0.9, 'learning_rate': 0.01, 'max_delta_step': 1, 'min_child_weight': 82, 'n_estimators': 666, 'subsample': 0.75, 'objective': 'multi:softmax', 'max_depth': 11, 'gamma': 2}

#Model with rank: 2
#Mean validation score: 0.577 (std: 0.001)
#Parameters: {'colsample_bytree': 1, 'learning_rate': 0.03, 'max_delta_step': 4, 'min_child_weight': 86, 'n_estimators': 877, 'subsample': 0.75, 'objective': 'multi:softmax', 'max_depth': 10, 'gamma': 8}

#Model with rank: 3
#Mean validation score: 0.569 (std: 0.003)
#Parameters: {'colsample_bytree': 0.5, 'learning_rate': 0.001, 'max_delta_step': 4, 'min_child_weight': 1, 'n_estimators': 384, 'subsample': 0.5, 'objective': 'multi:softmax', 'max_depth': 7, 'gamma': 2}






(59381, 126) (59381,)
RandomizedSearchCV took 112468.81 seconds for 50 candidates parameter settings.
Model with rank: 1
Mean validation score: 0.579 (std: 0.002)
Parameters: {'colsample_bytree': 0.9, 'learning_rate': 0.01, 'max_delta_step': 1, 'min_child_weight': 82, 'n_estimators': 666, 'subsample': 0.75, 'objective': 'multi:softmax', 'max_depth': 11, 'gamma': 2}

Model with rank: 2
Mean validation score: 0.577 (std: 0.001)
Parameters: {'colsample_bytree': 1, 'learning_rate': 0.03, 'max_delta_step': 4, 'min_child_weight': 86, 'n_estimators': 877, 'subsample': 0.75, 'objective': 'multi:softmax', 'max_depth': 10, 'gamma': 8}

Model with rank: 3
Mean validation score: 0.569 (std: 0.003)
Parameters: {'colsample_bytree': 0.5, 'learning_rate': 0.001, 'max_delta_step': 4, 'min_child_weight': 1, 'n_estimators': 384, 'subsample': 0.5, 'objective': 'multi:softmax', 'max_depth': 7, 'gamma': 2}



In [50]:
87973/60.0

1466.2166666666667

In [8]:
#an optimizer... need to look into how to use it properly, and what it does? bayesian?
import spearmint

In [9]:
#example code from scipy

import numpy as np

from time import time
from operator import itemgetter
from scipy.stats import randint as sp_randint

from sklearn.grid_search import GridSearchCV, RandomizedSearchCV
from sklearn.datasets import load_digits
from sklearn.ensemble import RandomForestClassifier

# get some data
digits = load_digits()
X, y = digits.data, digits.target

# build a classifier
clf = RandomForestClassifier(n_estimators=20)


# Utility function to report best scores
def report(grid_scores, n_top=3):
    top_scores = sorted(grid_scores, key=itemgetter(1), reverse=True)[:n_top]
    for i, score in enumerate(top_scores):
        print("Model with rank: {0}".format(i + 1))
        print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
              score.mean_validation_score,
              np.std(score.cv_validation_scores)))
        print("Parameters: {0}".format(score.parameters))
        print("")


# specify parameters and distributions to sample from
param_dist = {"max_depth": [3, None],
              "max_features": sp_randint(1, 11),
              "min_samples_split": sp_randint(1, 11),
              "min_samples_leaf": sp_randint(1, 11),
              "bootstrap": [True, False],
              "criterion": ["gini", "entropy"]}

# run randomized search
n_iter_search = 20
random_search = RandomizedSearchCV(clf, param_distributions=param_dist,
                                   n_iter=n_iter_search)

start = time()
random_search.fit(X, y)
print("RandomizedSearchCV took %.2f seconds for %d candidates"
      " parameter settings." % ((time() - start), n_iter_search))
report(random_search.grid_scores_)

# use a full grid over all parameters
param_grid = {"max_depth": [3, None],
              "max_features": [1, 3, 10],
              "min_samples_split": [1, 3, 10],
              "min_samples_leaf": [1, 3, 10],
              "bootstrap": [True, False],
              "criterion": ["gini", "entropy"]}

# run grid search
grid_search = GridSearchCV(clf, param_grid=param_grid)
start = time()
grid_search.fit(X, y)

print("GridSearchCV took %.2f seconds for %d candidate parameter settings."
      % (time() - start, len(grid_search.grid_scores_)))
report(grid_search.grid_scores_)

RandomizedSearchCV took 3.26 seconds for 20 candidates parameter settings.
Model with rank: 1
Mean validation score: 0.924 (std: 0.010)
Parameters: {'bootstrap': False, 'min_samples_leaf': 4, 'min_samples_split': 6, 'criterion': 'gini', 'max_features': 9, 'max_depth': None}

Model with rank: 2
Mean validation score: 0.921 (std: 0.019)
Parameters: {'bootstrap': False, 'min_samples_leaf': 3, 'min_samples_split': 9, 'criterion': 'entropy', 'max_features': 10, 'max_depth': None}

Model with rank: 3
Mean validation score: 0.918 (std: 0.015)
Parameters: {'bootstrap': False, 'min_samples_leaf': 9, 'min_samples_split': 3, 'criterion': 'entropy', 'max_features': 10, 'max_depth': None}

GridSearchCV took 32.67 seconds for 216 candidate parameter settings.
Model with rank: 1
Mean validation score: 0.932 (std: 0.006)
Parameters: {'bootstrap': False, 'min_samples_leaf': 1, 'min_samples_split': 1, 'criterion': 'gini', 'max_features': 10, 'max_depth': None}

Model with rank: 2
Mean validation score: 

In [25]:
from moe.easy_interface.experiment import Experiment


ImportError: No module named moe.easy_interface.experiment